# Wildfire Cosimulation Game

This notebook serves as an example of [Cosimulation](https://sedaro.github.io/openapi/#tag/Externals) in Sedaro. Here, Sedaro's external cosimulation interfaces have been applied to create an interactive video game involving the Wildfire demo available to each Sedaro user.

_This notebook utilizes newer Python features like the walrus operator (`:=`) from 3.8 and `match`-`case` statements from 3.10. If you are not using an up-to-date version of Python, you will need to make changes to parts of this notebook to account for the absence of newly-supported syntax._

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.


## Necessary Imports & Definitions

The following package imports, variables, functions, and classes will be necessary to execute various parts of this notebook.


### Sedaro Variables

You will need to update some of the string values below in your `config.json` file. Get the branch ID for the Wildfire scenario that you would like this notebook to target and input the ID as the Wildfire `SCENARIO_BRANCH_ID` variable. The `WILDFIRE_ID` string below should not require changing, but if you run into problems you may need to check this value to verify that it matches the agent ID of Wildfire in the targeted scenario. The `HOST` and `WEB_HOST` variables should only require updating if you are running a separate, specialized instance of the Sedaro software.

With all variables defined with their respective values, the code block below will call the Sedaro API and make `scenario` and `simulation` handles that will be used throughout this notebook.

#### API Key

This notebook requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


In [ ]:
import json
from sedaro import SedaroApiClient

with open('../secrets.json', 'r') as file:  # open API key file
    API_KEY = json.load(file)['API_KEY']  # read API key from file

with open('../config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
SCENARIO_BRANCH_ID = config['WILDFIRE']['SCENARIO_BRANCH_ID']                # ID of the scenario branch
WILDFIRE_ID = "NT06aqHUT5djI1_JPAsck"                                        # Wildfire ID (should not need changing)
HOST = config['HOST']                                                        # Sedaro instance URL
WEB_HOST = config['WEB_HOST']                                                # Sedaro web URL

In [ ]:
sedaro = SedaroApiClient(host=HOST, api_key=API_KEY)
scenario = sedaro.scenario(SCENARIO_BRANCH_ID)
simulation = scenario.simulation

### Simple Conversion Functions

Sedaro outputs the simulation time in MJD and angles in radians. These functions will help to convert MJD dates to `datetime` objects and radians to degrees for easier interpretation.


In [ ]:
from math import pi
from datetime import datetime, timedelta


def mjd2dt(mjd: float) -> datetime:
    """Convert Modified Julian Date (MJD) float to datetime object."""
    return datetime(1858, 11, 17) + timedelta(days=mjd)


def rad2deg(rad: float) -> float:
    """Convert radians to degrees."""
    return rad * 180 / pi

### Rotation & Quaternion Functions

Below are functions that will allow us to calculate the rotation matrix of a quaternion and convert between the spacecraft body frame and the Local-Vertical/Local-Horizontal (LVLH) frame. Expressing vectors with LVLH components will allow us to calculate yaw and pitch in an intuitive frame of reference.


In [ ]:
import numpy as np


def quaternion2RotMat(quaternion: np.ndarray) -> np.ndarray:
    """Convert quaternion to rotation matrix."""
    if len(quaternion) != 4:
        raise ValueError("Bad shape")

    rotation = np.zeros((3, 3))

    rotation[0, 0] = 1 - 2 * (quaternion[1] * quaternion[1] + quaternion[2] * quaternion[2])
    rotation[1, 0] = 2 * (quaternion[0] * quaternion[1] + quaternion[3] * quaternion[2])
    rotation[2, 0] = 2 * (quaternion[0] * quaternion[2] - quaternion[3] * quaternion[1])

    rotation[0, 1] = 2 * (quaternion[0] * quaternion[1] - quaternion[3] * quaternion[2])
    rotation[1, 1] = 1 - 2 * (quaternion[0] * quaternion[0] + quaternion[2] * quaternion[2])
    rotation[2, 1] = 2 * (quaternion[1] * quaternion[2] + quaternion[3] * quaternion[0])

    rotation[0, 2] = 2 * (quaternion[0] * quaternion[2] + quaternion[3] * quaternion[1])
    rotation[1, 2] = 2 * (quaternion[1] * quaternion[2] - quaternion[3] * quaternion[0])
    rotation[2, 2] = 1 - 2 * (quaternion[0] * quaternion[0] + quaternion[1] * quaternion[1])

    return rotation


def body2Lvlh(attitude, lvlh_to_eci):
    """Calculate body to LVLH rotation matrix."""
    body_to_eci = quaternion2RotMat(attitude)  # body to ECI rotation matrix
    eci_to_lvlh = lvlh_to_eci.T  # transpose for ECI to LVLH rotation matrix
    body_to_lvlh = eci_to_lvlh @ body_to_eci  # body to LVLH rotation matrix
    return body_to_lvlh

### Keyboard Input

This notebook utilizes the `pynput` package to interpret keypresses and trigger actions depending on which keys are pressed. These functions define the behavior of the `on_press` and `on_release` callbacks for the keyboard listener. The default keybindings for the `on_press` keypress callback are as follows:
| Key | Action |
| :-: | :-: |
| W | RW-Y forward, pitches down |
| S | RW-Y reverse, pitches up |
| A | RW-Z reverse, yaws left |
| D | RW-Z forward, yaws right |
| Q | RW-X reverse, rolls left |
| E | RW-X forward, rolls right |
| Spacebar | Burn thruster |
| 1–9 | Change active operational mode |
| Esc | Stop keyboard listener |

The `keyboard.Listener` object is initialized, but it will not be listening until it is started later.


In [ ]:
from pynput import keyboard


def on_press(key):
    global RWs, thruster, sat
    if isinstance(key, keyboard.KeyCode):
        match key.char:
            case 'w':
                RWs['Y'].forward()  # pitch down
            case 's':
                RWs['Y'].reverse()  # pitch up
            case 'a':
                RWs['Z'].reverse()  # yaw left
            case 'd':
                RWs['Z'].forward()  # yaw right
            case 'q':
                RWs['X'].reverse()  # roll left
            case 'e':
                RWs['X'].forward()  # roll right
            case n if int(n) in range(1, 10):
                sat.setRoutine(int(n))  # set routine
    elif isinstance(key, keyboard.Key):
        if key == keyboard.Key.space:
            thruster.burn()  # fire thruster
        elif key == keyboard.Key.esc:
            print('Stopping the keyboard listener.')
            return False  # stop listener


def on_release(key):
    global RWs, thruster
    if isinstance(key, keyboard.KeyCode):
        match key.char:
            case 'w' | 's':
                RWs['Y'].stop()
            case 'a' | 'd':
                RWs['Z'].stop()
            case 'q' | 'e':
                RWs['X'].stop()
    elif isinstance(key, keyboard.Key):
        if key == keyboard.Key.space:
            thruster.stop()


listener = keyboard.Listener(on_press=on_press, on_release=on_release)  # initialize keyboard listener

### Game Classes

These classes will help logically organize, calculate, and recall information during the game loop. They are separate from classes within the `sedaro` Python package; they are purely for game logic.

The different classes each handle important roles during the game loop:

- `Game`: stores the simulation time and game loop time and handles checking for a simulation update and waiting for the remainder of a game loop
- `Satellite`: stores various variables relevant to the satellite as a whole, handles setting the active routine, and calculates LVLH yaw and pitch from its current state
- `RW`: stores reaction wheel torque torque and momentum information and handles utilizing the reaction wheels
- `Thruster`: stores thruster attributes and handles turning the thruster on and off


In [ ]:
from time import sleep


class Game:
    def __init__(self, clock, min_loop_time=0.2):
        self.prevTime = clock.startTime  # initialize previous game loop time to start time
        self.minLoopTime = min_loop_time  # minimum game loop time [seconds]
        self.time = None
        self.loopStart = None
        self.loopEnd = None
        self.loopTime = None

    def startLoop(self):
        self.loopStart = datetime.now()

    def endLoop(self):
        self.loopEnd = datetime.now()
        self.loopTime = (self.loopEnd - self.loopStart).total_seconds()  # calculate loop time [seconds]

    def wait(self):
        if self.loopTime < self.minLoopTime:
            sleep(self.minLoopTime - self.loopTime)  # sleep for the remainder of the minimum game loop time

    def update(self):
        if self.prevTime < self.time:  # if previous time is less than current time (time has changed)
            self.prevTime = self.time  # update previous time to current time
            return True  # return True because time has changed
        else:
            return False  # return False because time has not changed


class Satellite:
    def __init__(self, routines):
        self.routines = routines
        self.activeRoutine = [routine for routine in routines if routine.name ==
                              "Standby"][0]  # set active routine to Standby
        self.stateOfCharge = None
        self.powerLoading = 0
        self.solarUtilization = None
        self.apogee = None
        self.perigee = None
        self.attitude = None
        self.lvlh2Eci = None
        self.yaw = None
        self.pitch = None

    def calcYawPitch(self, thruster):
        """Calculate the yaw and pitch angles of the thrust vector in the ram frame"""
        vector = -thruster.exhaustVector  # thrust vector in body frame
        vector = body2Lvlh(self.attitude, self.lvlh2Eci) @ vector  # vector in LVLH frame
        self.yaw = -np.arctan2(vector[2], vector[1])  # yaw angle from ram in LVLH frame
        self.pitch = np.arcsin(vector[0])  # pitch angle from ram in LVLH frame

    def setRoutine(self, num: int):
        """Set the active routine based on number"""
        routine_num = min(num, len(self.routines))  # limit input to number of routines
        self.activeRoutine = self.routines[routine_num - 1]  # set active routine


class RW:
    def __init__(self, ratedTorque: float, ratedMomentum: float):
        self.ratedTorque = ratedTorque
        self.torque = 0
        self.ratedMomentum = ratedMomentum
        self.momentum = 0

    def forward(self):
        self.torque = self.ratedTorque

    def reverse(self):
        self.torque = -self.ratedTorque

    def stop(self):
        self.torque = 0


class Thruster:
    def __init__(self, maxThrust: float, exhaustVector, capacity: float):
        self.maxThrust = maxThrust
        self.thrust = 0
        self.exhaustVector = exhaustVector
        self.capacity = capacity
        self.fuel = capacity

    def burn(self):
        self.thrust = self.maxThrust

    def stop(self):
        self.thrust = 0

### User Interface

These functions define the format and logic for displaying information during the game loop. `bar` generates a colored capacity bar that will help to visualize various variables. `display` formats information from the game classes and displays a user interface in the [Game Loop](#game-loop) output cell. The colors used by the `bar` and `display` functions are ANSI color codes and can be changed to use any valid color codes that you like.


In [ ]:
from IPython.display import clear_output


def bar(num: float, minimum: float, maximum: float, start: float = 0, width: int = 101):
    bar_color = '\033[34m'  # blue
    slide_color = '\033[36m'  # cyan
    rail_color = '\033[0m'  # reset

    num = max(minimum, min(num, maximum))  # keep num within bounds
    w = width - 2  # adjust width for brackets
    str = [slide_color + '-'] * w  # create list of slide characters
    i_start, i_num = [min(int((n - minimum) / (maximum - minimum) * w), w - 1)
                      for n in [start, num]]  # get indices of start and num
    # (inclusively) loop between start and num indices
    for i in range(i_start, i_num + 1, 1) if i_num > i_start else range(i_start, i_num - 1, -1):
        str[i] = bar_color + '\u2588'  # insert bar at index
    str[i_start] = rail_color + '|'  # insert start rail at start index
    str.insert(0, rail_color + '[')  # add left bracket
    str.append(rail_color + ']')  # add right bracket
    return ''.join(str)  # return joined list as string


def display(update: bool, game: Game, sat: Satellite, RWs: dict, thruster: Thruster):
    update_color = '\033[92m'  # green
    bold = '\033[1m'  # bold
    reset = '\033[0m'

    info_string = "\n".join([
        f"{bold}Loop Time{reset}:           {game.loopTime:.4f} seconds",
        "",
        f"{bold}Simulation Time{reset}:     {update_color if update else ''}{mjd2dt(game.time)}{reset}",
        "",
        f"{bold}Routine{reset}:             [{sat.routines.index(sat.activeRoutine) + 1}] {sat.activeRoutine.name}",
        "",
        f"{bold}---- Orbit ----{reset}",
        f"  Apogee:            {sat.apogee:9.4f} km",
        f"  Perigee:           {sat.perigee:9.4f} km",
        "",
        f"{bold}---- Attitude ----{reset}",
        f"  Yaw from Ram:      {bar(rad2deg(sat.yaw), -180, 180)} {rad2deg(sat.yaw):7.2f}\u00B0",
        f"  Pitch from Ram:    {bar(rad2deg(sat.pitch), -90, 90)} {rad2deg(sat.pitch):7.2f}\u00B0",
        "",
        f"{bold}---- Actuators ----{reset}",
        f"                          {bold}Actuation{reset}      |                                                {bold}Capacity{reset}",
        f"  RW X:              {bar(RWs['X'].torque, -RWs['X'].ratedTorque, RWs['X'].ratedTorque, 0, 9)} {RWs['X'].torque:5.2f} N\u22C5m | {bar(RWs['X'].momentum, -RWs['X'].ratedMomentum, RWs['X'].ratedMomentum)} {RWs['X'].momentum:5.2f} N\u22C5s",
        f"  RW Y:              {bar(RWs['Y'].torque, -RWs['Y'].ratedTorque, RWs['Y'].ratedTorque, 0, 9)} {RWs['Y'].torque:5.2f} N\u22C5m | {bar(RWs['Y'].momentum, -RWs['Y'].ratedMomentum, RWs['Y'].ratedMomentum)} {RWs['Y'].momentum:5.2f} N\u22C5s",
        f"  RW Z:              {bar(RWs['Z'].torque, -RWs['Z'].ratedTorque, RWs['Z'].ratedTorque, 0, 9)} {RWs['Z'].torque:5.2f} N\u22C5m | {bar(RWs['Z'].momentum, -RWs['Z'].ratedMomentum, RWs['Z'].ratedMomentum)} {RWs['Z'].momentum:5.2f} N\u22C5s",
        f"  Thruster:          {bar(thruster.thrust, 0, thruster.maxThrust, 0, 9)} {thruster.thrust:5.1f} N   | {bar(thruster.fuel, 0, thruster.capacity)} {thruster.fuel:5.2f} kg",
        "",
        f"{bold}---- Power ----{reset}",
        f"  Power Loading:     {sat.powerLoading:5.2f} W",
        f"  Battery Charge:    {bar(sat.stateOfCharge, 0, 1)} {sat.stateOfCharge:7.2%}",
        f"  Solar Utilization: {bar(sat.solarUtilization, 0, 1)} {sat.solarUtilization:7.2%}",
    ])
    clear_output()  # clear output of Jupyter Notebook cell
    print(info_string, flush=True)

## Scenario Setup

Now that the relevant functions and classes have been defined, we need to edit specific scenario blocks and initialize instances of the game classes with information gathered from Wildfire's agent template.


### External State Block

To query and change the current state of variables during the game loop while the simulation is running, this notebook will make use of External State blocks. External State blocks are defined at the scenario level and allow external sources to read from and write to an agent's associated variables within the simulation. There are two kinds of External State blocks:

- `PerRoundExternalState`: reads/writes state from an agent for each step of the simulation, with the simulation being blocked until each `PerRoundExternalState` block has consumed and produced its associated variables for that simulation time step
- `SpontaneousExternalState`: reads/writes state from an agent whenever called, with the simulation not being blocked if the `SpontaneousExternalState` block is not called

Each External State block must define the variables that it intends to read and/or write with the `consume` and `produce` fields. It must also be linked to a specific Agent Block within the scenario via the `agent` relationship. The simulation engine that controls its associated variables must also be specified via the `engineIndex` field. Each `engineIndex` value corresponds to a different engine:

0.  Guidance, Navigation, & Control
1.  Command & Data Handling
2.  Power
3.  Thermal

For more information on Externals and cosimulation in Sedaro, please see the docs [here](https://sedaro.github.io/openapi/#tag/Externals).


Before creating the External State blocks for this notebook, it is helpful to ensure that there are no existing External State blocks associated with the target scenario. The code block below will find all existing `ExternalState` blocks and remove them.


In [ ]:
if len(extStateIDs := scenario.ExternalState.get_all_ids()) > 0:  # if there are any existing ExternalState blocks
    scenario.crud(delete=extStateIDs)  # delete all ExternalState blocks

After deleting any residual `ExternalState` blocks from the scenario, we can create the blocks that will facilitate simulation input and output for the variables relevant to the game. Since we are not driving the simulation's time steps and do not need our game loop to be tied to simulation steps, we will use `SpontaneousExternalState` blocks. Below we create 3 different `SpontaneousExternalState` blocks for the Wildfire agent associated with 3 different simulation engines.


In [ ]:
gnc_state_block = scenario.SpontaneousExternalState.create(**{
    'consumed': [
        "time",
        {"root": "attitude"},
        {"root": "lvlhAxes"},
        {"ReactionWheel": "momentum"},
        {"FuelTank": "wetMass"},
        {"Orbit": "radiusApogee"},
        {"Orbit": "radiusPerigee"}],
    'produced': [
        {"ReactionWheel": "commandedTorqueMagnitude"},
        {"Thruster": "thrust"}],
    'engineIndex': 0,
    'agents': [WILDFIRE_ID],
}
)
GNC_STATE_ID = gnc_state_block.id
cdh_state_block = scenario.SpontaneousExternalState.create(**{
    'produced': [{"CombinationalLogic": "activeSubroutine"}],
    'engineIndex': 1,
    'agents': [WILDFIRE_ID],
}
)
CDH_STATE_ID = cdh_state_block.id
power_state_block = scenario.SpontaneousExternalState.create(**{
    'consumed': [
        {"Battery": "soc"},
        {"PowerProcessor": "outputPowers"},
        {"SolarArray": "utilization"}],
    'engineIndex': 2,
    'agents': [WILDFIRE_ID],
}
)
POWER_STATE_ID = power_state_block.id

### Clock Configuration

By default, the scenario clock is configured to not limit the speed of the simulation. To make our game more interactive, we set the `ClockConfig` to run in real-time. This only needs to be done once as every subsequent simulation will now run in real-time until this setting is changed back to `realTime=False`.


In [ ]:
clock_config = scenario.ClockConfig.get_first()  # get clock config block
clock_config.update(realTime=True, syncTime=False)  # set real-time mode with syncTime off

### Game Class Initialization

The following code blocks will initialize the game class objects with information gathered from the Wildfire agent template. The template will provide information on the reaction wheels, thruster, and operational modes that are necessary for the game's behavior.


In [ ]:
agent_block = scenario.block(WILDFIRE_ID)
AGENT_TEMPLATE_ID = agent_block.templateRef  # get referenced agent template branch ID
agent_template = sedaro.agent_template(AGENT_TEMPLATE_ID)  # get agent template

#### Actuator Initialization

We initialize the reaction wheel objects using the maximum torque and momentum for each corresponding `ReactionWheel` block on the agent template. The thruster object is initialized with the maximum thrust, exhaust body frame vector, and fuel tank capacity associated with the `Thruster` block on the agent template.

_If you have changed the names (or orientations) of Wildfire's reaction wheels, you will need to update the `RW_names` dictionary in the block below as it uses these name values to identify each `ReactionWheel` block._


In [ ]:
# Reaction Wheels
RW_names = {
    'X': "RW-X",
    'Y': "RW-Y",
    'Z': "RW-Z"
}  # reaction wheel names for their corresponding axes
RWs = {}
reaction_wheel_IDs = agent_template.data['index']['ReactionWheel']  # list of reaction wheel IDs in agent template
for rw_ID in reaction_wheel_IDs:  # iterate through reaction wheels to get rated torques
    rw_block = agent_template.block(rw_ID)  # reaction wheel block
    for axis, name in RW_names.items():
        if rw_block.name == name:
            RWs[axis] = RW(ratedTorque=rw_block.ratedTorque,
                           ratedMomentum=rw_block.ratedMomentum)  # initialize reaction wheel
            break

# Thruster
thruster_ID = agent_template.data['index']['Thruster'][0]  # thruster ID in agent template
thruster_block = agent_template.block(thruster_ID)  # thruster block
thruster = Thruster(maxThrust=thruster_block.maxThrust,  # maximum thrust of thruster
                    exhaustVector=np.array(thruster_block.orientation.unitVector),  # thruster exhaust vector
                    # capacity of thruster's fuel tanks
                    capacity=sum(tank.capacity for tank in thruster_block.fuelReservoir.fuelTanks),
                    )  # initialize thruster

#### Routines

To define which routines should be options for switching the active routine during the game, we query for all of the `Routine` blocks in the Wildfire agent template, find the first one named "Root Routine", and then get its subroutines. A routine will be selected with a corresponding number key and the root routine's `activeSubroutine` state will be set to that routines's ID to change the active subroutine within the simulation.


In [ ]:
routines = agent_template.Routine.get_all()  # get all routine blocks
root_routine = [routine for routine in routines if routine.name == "Default Main Routine"][0]  # get root routine block
subroutine_blocks = [routine for routine in routines if routine in root_routine.subroutines]  # get subroutine blocks

#### Game & Satellite Initialization

The `Game` class object is now created with the Scenario's `ClockConfig` block passed to it and the `Satellite` class object is initialized with the list of subroutine blocks. This is the last necessary step in laying the foundation for the game loop to run.


In [ ]:
game = Game(clock_config)  # initialize game object
sat = Satellite(subroutine_blocks)  # initialize satellite object with subroutines

## Running the Game

Now that necessary variables and functions have been defined, scenario blocks have been configured, and game classes have been initialized with information from the agent template, we can finally deploy the simulation and begin the game loop.


### Starting Simulation & Listener

These blocks will start the simulation and the keyboard listener. Starting the simulation can take longer for more complicated scenarios, so you may need to give it some time (~30 seconds). After the simulation is started, the code block below will print a link that you can use to view the scenario results in a web browser.


In [ ]:
# start the simulation and wait for it to initialize (this will take a few seconds)
simulation_handle = simulation.start(wait=True)
# print link to scenario results
print(
    f"View the scenario results at {WEB_HOST}/#/agent-analyze/{SCENARIO_BRANCH_ID}/custom/playback?agentId={WILDFIRE_ID}")

In [ ]:
listener.start()  # start non-blocking keyboard listener in another thread

### Game Loop

This block contains the game loop, which will continue to run while the keyboard listener is active and the simulation is running. The game loop performs the following steps during each iteration:

1.  Start the game loop timer
2.  Set the simulation update variable (`sim_updated`) to `False`
3.  Consume & produce the simulation state defined by the `SpontaneousExternalState` blocks
4.  Unpack the simulation state tuples
5.  Update game class attributes
6.  Detect if the GNC engine time has updated
7.  End game loop timer
8.  Update display
9.  If necessary, wait for the time remaining in the minimum game time loop

If the game loop is completed in less time than defined by the `minLoopTime` attribute of the `Game` class, the loop will wait for the remainder of the minimum loop time and then begin the next loop iteration. The default controls are listed in the [Keyboard Input](#keyboard-input) section. The user interface outlined in the [User Interface](#user-interface) section will be displayed as the output of the game loop cell below. By default, the keyboard listener can be stopped (therefore ending the game loop) by pressing the Esc key.


In [ ]:
while listener.running and simulation_handle.status()['status'] == 'RUNNING':
    game.startLoop()  # start Game loop timer
    sim_updated = False  # set simulation updated flag to False at the start of each loop

    # Consume & produce simulation state
    # query the simulation for the current GNC state
    gnc_state = simulation_handle.consume(agent_id=WILDFIRE_ID, external_state_id=GNC_STATE_ID)
    simulation_handle.produce(agent_id=WILDFIRE_ID,
                              external_state_id=GNC_STATE_ID,
                              values=(
                                  [RWs['X'].torque, RWs['Y'].torque, RWs['Z'].torque],
                                  [thruster.thrust]))
    simulation_handle.produce(agent_id=WILDFIRE_ID,
                              external_state_id=CDH_STATE_ID,
                              values=([sat.activeRoutine.id,],))
    # query the simulation for the current Power state
    power_state = simulation_handle.consume(agent_id=WILDFIRE_ID, external_state_id=POWER_STATE_ID)

    # Parse simulation output
    (
        time,  # simulation time [MJD]
        attitude,  # attitude quaternion
        lvlhAxes,  # LVLH to ECI rotation matrix
        momentum,  # reaction wheel momentum list
        wetMass,  # tank fuel list [kg]
        apogee,  # apogee list [km]
        perigee  # perigee list [km]
    ) = gnc_state  # unpack the GNC state tuple
    (
        stateOfCharge,  # battery state of charge list
        powerLoading,  # power processor output power list
        solarUtilization,  # solar array utilization list
    ) = power_state  # unpack the power state tuple

    # Update fields
    game.time = time  # update game time
    sat.attitude = attitude  # update attitude quaternion
    sat.lvlh2Eci = lvlhAxes  # update LVLH axes
    sat.calcYawPitch(thruster)  # calculate yaw and pitch angles
    for rw, m in zip(RWs.values(), momentum):
        rw.momentum = m  # update reaction wheel momentum
    thruster.fuel = sum(wetMass)  # update thruster fuel
    sat.apogee = apogee[0]  # update apogee
    sat.perigee = perigee[0]  # update perigee
    sat.stateOfCharge = stateOfCharge[0]
    sat.powerLoading = powerLoading[0]['total']  # update total power loading
    totalUtilization = sum([(util if not np.isnan(util) else 0)
                           for util in solarUtilization])  # sum utilization values, ignoring NaNs
    sat.solarUtilization = totalUtilization / len(solarUtilization)  # update average solar array utilization

    sim_updated = game.update()  # detect if the simulation time has changed & update Game clock
    game.endLoop()  # end Game loop timer
    display(sim_updated, game, sat, RWs, thruster)  # update display output
    game.wait()  # wait if necessary

### Stopping Simulation & Listener

If they have not already stopped running after ending the game loop, the two blocks below will stop the keyboard listener and terminate the simulation.


In [ ]:
listener.stop()  # stop keyboard listener if it hasn't been stopped already

In [ ]:
try:
    simulation_handle.terminate()  # stop simulation if it hasn't stopped already
except:
    pass